In [48]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
import time

In [63]:
class SongCreator:
    def __init__(self, url = "https://www.billboard.com/charts/hot-100", path = "./songs/not_hot.csv"):
        self.top100 = self.scrape_hot100(url)
        self.not_hot = pd.read_csv(path)
        self.main = pd.DataFrame()

    def __str__(self):
        return pd.concat([self.top100, self.not_hot]).to_string(index=False)

    def CreateMain(self):
        try:
            self.not_hot = self.not_hot[self.top100.columns]
            self.main = pd.concat([self.top100, self.not_hot])
        except:
            raise ValueError ("Make sure both dataframes have equal columns")

    def DropColumns(self, drop = list) -> None:
        tuple = (self.top100.columns, self.not_hot.columns)
        for x, y in tuple:
            if x in drop:
                self.top100 = self.top100.drop(x, axis=1)
            if y in drop:
                self.not_hot = self.not_hot.drop(y, axis=1)

    def to_csv(self, filename):
        try:
            self.createmain()
            self.main.to_csv(filename, index=False)
        except:
            pass

    def scrape_hot100(self, url):
        response = requests.get(url)
        if response.status_code != 200:
            raise ValueError("Error connecting to the website")

        soup = BeautifulSoup(response.content, "html.parser")

        titles = [elem.get_text().replace('\n', '').replace('\t', '') for elem in soup.select("div.chart-results-list div.o-chart-results-list-row-container li.lrv-u-width-100p h3")]
        artists = [elem.get_text().replace('\n', '').replace('\t', '') for elem in soup.select("div.chart-results-list div.o-chart-results-list-row-container li.lrv-u-width-100p span.u-letter-spacing-0021")]

        return pd.DataFrame({"Title": titles, "Artist": artists, "Rank": range(1, 101)})

    def SearchBulk(client: SpotifyClientCredentials, tracks:list, chunk_size=5) -> pd.DataFrame:
        song_names = []
        song_ids = []
        artists = []

        for i in range(0, len(tracks)):
            try:
                response = client.search(tracks[i], limit=1)
                song_names.append(response['tracks']['items'][0]['name'])
                song_ids.append(response['tracks']['items'][0]['id'])
                artists.append(response['tracks']['items'][0]['artists'][0]['name'])

            except BaseException as err:
                print(f'Error processing tracks ({i}) -> {err}')
            
            if i % chunk_size == 0:
                time.sleep(1)
                print(f'({i}/{len(tracks)}) {round((i/len(tracks)*100),1)}%')
    
        return pd.DataFrame({"id":song_ids, "name":song_names, "artists": artists})
    
    def GetCredentials():
        pass

    def GetAudioFeatures(client: SpotifyClientCredentials, df: pd.DataFrame, columnID='id', chunk_size=100) -> pd.DataFrame:
        
        completed = 0
        tracks = df[columnID].to_list()
        result_df = pd.DataFrame()

        for i in range(0, len(tracks), chunk_size):
            try:
                responses = client.audio_features(tracks=tracks[i:i+chunk_size])
                
                for response in responses:
                    dftemp = pd.DataFrame([response])
                    result_df = pd.concat([result_df, dftemp], ignore_index=True)

            except BaseException as err:
                print(f'Error processing tracks ({i} : {i+chunk_size}) -> {err}')
            
            completed += chunk_size
            print(f'({completed}/{len(tracks)}) {round((completed/len(tracks)*100),1)}%')
            time.sleep(1)

        return pd.merge(df, result_df, on=columnID)

In [57]:
Song = SongCreator()

In [59]:
Song.not_hot

,Unnamed: 0.1,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,uri,track_href,analysis_url,duration_ms,time_signature,genre,song_name,Unnamed: 0,title,Artists
0,0,0.831,0.814,2,-7.364,1,0.4200,0.05980,0.013400,0.0556,...,spotify:track:2Vc6NJ9PW9gD9q343XFRKx,https://api.spotify.com/v1/tracks/2Vc6NJ9PW9gD...,https://api.spotify.com/v1/audio-analysis/2Vc6...,124539,4,Dark Trap,Mercury: Retrograde,NaN,NaN,Ghostemane
1,1,0.719,0.493,8,-7.230,1,0.0794,0.40100,0.000000,0.1180,...,spotify:track:7pgJBLVz5VmnL7uGHmRj6p,https://api.spotify.com/v1/tracks/7pgJBLVz5Vmn...,https://api.spotify.com/v1/audio-analysis/7pgJ...,224427,4,Dark Trap,Pathology,NaN,NaN,Don Kenobi
2,2,0.850,0.893,5,-4.783,1,0.0623,0.01380,0.000004,0.3720,...,spotify:track:0vSWgAlfpye0WCGeNmuNhy,https://api.spotify.com/v1/tracks/0vSWgAlfpye0...,https://api.spotify.com/v1/audio-analysis/0vSW...,98821,4,Dark Trap,Symbiote,NaN,NaN,gizmo
3,3,0.476,0.781,0,-4.710,1,0.1030,0.02370,0.000000,0.1140,...,spotify:track:0VSXnJqQkwuH2ei1nOQ1nu,https://api.spotify.com/v1/tracks/0VSXnJqQkwuH...,https://api.spotify.com/v1/audio-analysis/0VSX...,123661,3,Dark Trap,ProductOfDrugs (Prod. The Virus and Antidote),NaN,NaN,Kamiyada+
4,4,0.798,0.624,2,-7.668,1,0.2930,0.21700,0.000000,0.1660,...,spotify:track:4jCeguq9rMTlbMmPHuO7S3,https://api.spotify.com/v1/tracks/4jCeguq9rMTl...,https://api.spotify.com/v1/audio-analysis/4jCe...,123298,4,Dark Trap,Venom,NaN,NaN,$uicideboy$
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2995,0.644,0.301,10,-8.927,0,0.0308,0.03160,0.856000,0.0919,...,spotify:track:1YiFZjmohJD78nlsSjjHGT,https://api.spotify.com/v1/tracks/1YiFZjmohJD7...,https://api.spotify.com/v1/audio-analysis/1YiF...,250697,4,Dark Trap,Focus,NaN,NaN,Tomkillsjerry
2996,2996,0.603,0.555,8,-10.284,0,0.0277,0.01680,0.907000,0.1070,...,spotify:track:7o5E34q2K5cpQfy2P1WEGW,https://api.spotify.com/v1/tracks/7o5E34q2K5cp...,https://api.spotify.com/v1/audio-analysis/7o5E...,219168,4,Dark Trap,The Cure,NaN,NaN,Tomkillsjerry
2997,2997,0.626,0.367,8,-11.263,1,0.0343,0.00349,0.956000,0.3730,...,spotify:track:0Wdp9TyN3PqCoxSzH8vRwO,https://api.spotify.com/v1/tracks/0Wdp9TyN3PqC...,https://api.spotify.com/v1/audio-analysis/0Wdp...,189806,4,Dark Trap,Blocks,NaN,NaN,Tomkillsjerry
2998,2998,0.616,0.572,11,-9.371,0,0.0755,0.01100,0.921000,0.1440,...,spotify:track:2K7NWYBbqGzNLggH7LIhxy,https://api.spotify.com/v1/tracks/2K7NWYBbqGzN...,https://api.spotify.com/v1/audio-analysis/2K7N...,247928,5,Dark Trap,Liu Kang,NaN,NaN,Tomkillsjerry
